# Question 1
## Find the longest conversation chain in the chat data using the "ResponseTo" edge label.

### How many chats are involved in it?

In [ ]:
MATCH p=()-[:ResponseTo*]->()
RETURN length(p) as length
ORDER BY length DESC
LIMIT 1

### How many users participated in this chain?

In [ ]:
# Users that Created or were mentioned in a ChatItem
MATCH p=(i1:ChatItem)-[:ResponseTo*]->(i2:ChatItem)
WITH max(length(p)) as max_length
MATCH p=(i1:ChatItem)-[:ResponseTo*]->(i2:ChatItem) WHERE length(p) = max_length
WITH [i in nodes(p)] as items
MATCH path=(u:User)-[:CreateChat]->(i:ChatItem)
WHERE i IN items
RETURN DISTINCT path

In [ ]:
MATCH p=(i1:ChatItem)-[:ResponseTo*]->(i2:ChatItem) WHERE length(p) = 9 
WITH [i in nodes(p)] as items
MATCH path=(u:User)-[:CreateChat]->(i:ChatItem)
WHERE i IN items
RETURN COUNT(DISTINCT u) AS Participating_Users

# Question 2:
## Do the top 10 the chattiest users belong to the top 10 chattiest teams? 

### Identify the top 10 chattiest users

In [ ]:
MATCH (u:User)-[c:CreateChat]->()
RETURN u.id as UserID, COUNT(c) as Chats
ORDER BY Chats DESC
LIMIT 10

```
╒════════╤═══════╕
│"UserID"│"Chats"│
╞════════╪═══════╡
│394     │115    │
├────────┼───────┤
│2067    │111    │
├────────┼───────┤
│1087    │109    │
├────────┼───────┤
│209     │109    │
├────────┼───────┤
│554     │107    │
├────────┼───────┤
│1627    │105    │
├────────┼───────┤
│516     │105    │
├────────┼───────┤
│999     │105    │
├────────┼───────┤
│668     │104    │
├────────┼───────┤
│461     │104    │
└────────┴───────┘
```

### Identify the top 10 chattiest teams

In [ ]:
MATCH (i:ChatItem)-[:PartOf]->(:TeamChatSession)-[:OwnedBy]->(t:Team)
RETURN t.id as TeamID, COUNT(i) as Chats
ORDER BY Chats DESC
LIMIT 10

```
╒════════╤═══════╕
│"TeamID"│"Chats"│
╞════════╪═══════╡
│82      │1324   │
├────────┼───────┤
│185     │1036   │
├────────┼───────┤
│112     │957    │
├────────┼───────┤
│18      │844    │
├────────┼───────┤
│194     │836    │
├────────┼───────┤
│129     │814    │
├────────┼───────┤
│52      │788    │
├────────┼───────┤
│136     │783    │
├────────┼───────┤
│146     │746    │
├────────┼───────┤
│81      │736    │
└────────┴───────┘
```

### If any of the chattiest users are part of the chattiest teams or not

In [ ]:
MATCH (u:User)-[c:CreateChat]->() 
WITH u, COUNT(c) as UserChats 
ORDER BY UserChats DESC LIMIT 10 
WITH [u.id] as ChattiestUsers

MATCH (u:User)-[:CreateChat]->(:ChatItem)-[:PartOf]->(:TeamChatSession)-[:OwnedBy]->(t:Team)
WHERE u.id IN ChattiestUsers
  AND t.id IN [82,185,112,18,194,129,52,136,146,81]
return DISTINCT u.id AS User, t.id as Team

```
╒══════╤══════╕
│"User"│"Team"│
╞══════╪══════╡
│999   │52    │
└──────┴──────┘
```

# Question 3:
## How Active are Groups of Users?

We will construct the neighborhood of users. In this neighborhood, we will connect two users if 
 - One user mentioned another user in a chat 
 - One user created a chatItem in response to another user’s chatItem

For the first condition, this query would have the following structure: 

```Match (u1:User)-SOMETHING IN THE MIDDLE-[:Mentioned]->(u2:User) create (u1)-[:InteractsWith]->(u2)```

In [ ]:
MATCH (u1:User)-[:CreateChat]->(:ChatItem)-[:Mentioned]->(u2:User)
CREATE (u1)-[:InteractsWith]->(u2)

Use the same logic to create the query statement for the second condition. This query will also have the form  
```MATCH PORTION followed create (u1)-[:InteractsWith]->(u2)```

In [ ]:
MATCH (u1:User)-[:CreateChat]->(:ChatItem)-[:ResponseTo]->(:ChatItem)<-[:CreateChat]-(u2:User)
CREATE (u1)-[:InteractsWith]->(u2)

The above scheme will create an undesirable side effect if a user has responded to their own chatItem, because it will create a self loop between two users.  
So after the first two steps we need to eliminate all self loops involving the edge “Interacts with”. This will take the form:  
```MATCH (u1)-[r:InteractsWith]->(u1) DELETE r```

In [ ]:
MATCH (u:User)-[c:CreateChat]->() 
WITH u, COUNT(c) as Chats 
ORDER BY Chats DESC LIMIT 10
WITH [u] as ChattiestUsers

//Getting the neighbours of all Users and the count
MATCH (u1:User)-[:InteractsWith]->(u2:User)
WHERE u1 in ChattiestUsers
WITH u1.id AS UserID, COLLECT(DISTINCT u2.id) AS Neighbours 
RETURN UserID, Neighbours, SIZE(Neighbours) AS k

```
╒════════╤═════════════════════════════════════════════╤═══╕
│"UserID"│"Neighbours"                                 │"k"│
╞════════╪═════════════════════════════════════════════╪═══╡
│394     │[1997,1012,2011,1782]                        │4  │
├────────┼─────────────────────────────────────────────┼───┤
│2067    │[697,1672,63,209,1627,516,1265,2096]         │8  │
├────────┼─────────────────────────────────────────────┼───┤
│1087    │[426,772,929,1879,1311,1098]                 │6  │
├────────┼─────────────────────────────────────────────┼───┤
│209     │[516,63,2067,1672,2096,1265,1627]            │7  │
├────────┼─────────────────────────────────────────────┼───┤
│554     │[2018,1687,1010,1959,1096,1412,610]          │7  │
├────────┼─────────────────────────────────────────────┼───┤
│1627    │[2096,1672,2067,63,516,209,697,1265]         │8  │
├────────┼─────────────────────────────────────────────┼───┤
│516     │[63,209,1672,2067,2096,1627,1265]            │7  │
├────────┼─────────────────────────────────────────────┼───┤
│999     │[1554,1587,778,1056,1606,1601,1398,1506,1839]│9  │
├────────┼─────────────────────────────────────────────┼───┤
│668     │[2034,648,698,458]                           │4  │
├────────┼─────────────────────────────────────────────┼───┤
│461     │[1675,1482,482]                              │3  │
└────────┴─────────────────────────────────────────────┴───┘
```

In [ ]:
// Getting TOP 10 Chattiest Users
MATCH (u:User)-[c:CreateChat]->() 
WITH u, COUNT(c) as Chats 
ORDER BY Chats DESC LIMIT 10
WITH [u] as ChattiestUsers

// Getting the neighbours of TOP 10 Users and the count
MATCH (u1:User)-[:InteractsWith]->(u2:User)
WHERE u1 in ChattiestUsers
WITH u1.id AS UserID, COLLECT(DISTINCT u2.id) AS Neighbours 
WITH UserID, Neighbours, SIZE(Neighbours) AS k

// Find Interecting Users
MATCH (u1:User)-[:InteractsWith]->(u2:User)
// Such that both belong in Neighbours list
WHERE u1.id IN Neighbours AND u2.id IN Neighbours
// For every valid combination of a User and its two neighbours, Value is 1 if neighbours have interacted atleast once, k is no. of Neighbours
WITH DISTINCT UserID, u1.id AS N1, u2.id as N2, CASE WHEN (u1)-[:InteractsWith]->(u2) THEN 1 ELSE 0 END AS VALUE, k

RETURN UserID,SUM(VALUE) as NUM,k*(k-1) AS DENUM, SUM(VALUE)/(k*(k-1)*1.0) AS ClusteringCoefficient
ORDER BY ClusteringCoefficient DESC

```
╒════════╤═════╤═══════╤═══════════════════════╕
│"UserID"│"NUM"│"DENUM"│"ClusteringCoefficient"│
╞════════╪═════╪═══════╪═══════════════════════╡
│668     │12   │12     │1.0                    │
├────────┼─────┼───────┼───────────────────────┤
│461     │6    │6      │1.0                    │
├────────┼─────┼───────┼───────────────────────┤
│209     │38   │42     │0.9047619047619048     │
├────────┼─────┼───────┼───────────────────────┤
│516     │38   │42     │0.9047619047619048     │
├────────┼─────┼───────┼───────────────────────┤
│394     │10   │12     │0.8333333333333334     │
├────────┼─────┼───────┼───────────────────────┤
│1087    │23   │30     │0.7666666666666667     │
├────────┼─────┼───────┼───────────────────────┤
│999     │55   │72     │0.7638888888888888     │
├────────┼─────┼───────┼───────────────────────┤
│554     │31   │42     │0.7380952380952381     │
├────────┼─────┼───────┼───────────────────────┤
│2067    │39   │56     │0.6964285714285714     │
├────────┼─────┼───────┼───────────────────────┤
│1627    │39   │56     │0.6964285714285714     │
└────────┴─────┴───────┴───────────────────────┘
```